In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
len(df)

3214

In [5]:
#Características
features = df.drop(['is_ultra'], axis=1)

In [6]:
#Objetivo
target = df['is_ultra']

In [7]:
#Segmentación en conjunto de entrenamiento (60%) y un conjunto temporal (40%)
features_train, features_temp, target_train, target_temp = train_test_split(features, target, test_size=0.4, random_state=12345)

In [8]:
print(len(features_train), len(target_train))

1928 1928


In [9]:
#Segmentación del conjunto temporal en uno de validación (50%) y uno de prueba (50%)
features_valid, features_test, target_valid, target_test = train_test_split(features_temp, target_temp, test_size=0.5, random_state=12345)

In [10]:
print(len(features_valid), len(target_valid), len(features_test), len(target_test))

643 643 643 643


### Calidad de diferentes modelos

#### Árbol de decisión

In [11]:
for depth in range(1, 11):    
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)

    predictions_train = model.predict(features_train)
    accuracy_train = accuracy_score(target_train, predictions_train)

    score_train = model.score(features_train, target_train)

    predictions_valid = model.predict(features_valid)
    accuracy_valid = accuracy_score(target_valid, predictions_valid)
    
    score_valid = model.score(features_valid, target_valid)

    print(f"max_depth = {depth}, score de entrenamiento = {score_train}, score de validación = {score_valid}")

max_depth = 1, score de entrenamiento = 0.7577800829875518, score de validación = 0.7542768273716952
max_depth = 2, score de entrenamiento = 0.7878630705394191, score de validación = 0.7822706065318819
max_depth = 3, score de entrenamiento = 0.8075726141078838, score de validación = 0.7853810264385692
max_depth = 4, score de entrenamiento = 0.8106846473029046, score de validación = 0.7791601866251944
max_depth = 5, score de entrenamiento = 0.8200207468879668, score de validación = 0.7791601866251944
max_depth = 6, score de entrenamiento = 0.8376556016597511, score de validación = 0.7838258164852255
max_depth = 7, score de entrenamiento = 0.8558091286307054, score de validación = 0.7822706065318819
max_depth = 8, score de entrenamiento = 0.8625518672199171, score de validación = 0.7791601866251944
max_depth = 9, score de entrenamiento = 0.8812240663900415, score de validación = 0.7822706065318819
max_depth = 10, score de entrenamiento = 0.8890041493775933, score de validación = 0.774494

#### Bosque aleatorio

In [12]:
best_score = 0
best_est = 0

for est in range (1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)

    score_valid = model.score(features_valid, target_valid)

    if score_valid > best_score:
        best_score = score_valid
        best_est = est

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 10): 0.7853810264385692


In [13]:
##Agregando profundidad

best_score = 0
best_est = 0
depth = 0

for est in range (1, 21):
    for depth in range (1, 21):
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)

        score_valid = model.score(features_valid, target_valid)

        if score_valid > best_score:
            best_score = score_valid
            best_est = est
            best_depth = depth
        #print("est = {}, depth = {}, score = {}".format(est, depth, score_valid))

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}, depth = {}): {}".format(best_est, best_depth, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 17, depth = 12): 0.8055987558320373


#### Regresión logística

In [14]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)

score_train = model.score(features_train, target_train)
score_valid = model.score(features_valid, target_valid)

print(f"Accuracy modelo de entrenamiento: {score_train}")
print(f"Accuracy modelo de validacion: {score_valid}")

Accuracy modelo de entrenamiento: 0.7157676348547718
Accuracy modelo de validacion: 0.7091757387247278


El modelo de bosque aleatorio con estimador de 10 tiene precisión del 78.54% al igual que el model de árbol de decisión con una profundidad de 3. El modelo de bosque aleatorio puede tener una mejor calidad si además variamos con la profundidad. Por esta razón voy a escoger este modelo para su comprobación.

### Comprobación de la calidad del modelo

In [15]:
best_score = 0
best_est = 0

for est in range (1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)

    score_test = model.score(features_test, target_test)

    if score_test > best_score:
        best_score = score_test
        best_est = est

print("La exactitud del mejor modelo en el conjunto de validación (n_estimators = {}): {}".format(best_est, best_score))

La exactitud del mejor modelo en el conjunto de validación (n_estimators = 8): 0.7869362363919129


El modelo tiene una precisión de 78.69% con el conjunto de prueba usando 8 estimadores.

### Prueba de cordura

In [16]:
##Dummy Classifier
from sklearn.dummy import DummyClassifier

dummy = DummyClassifier(strategy='most_frequent', random_state=12345)
dummy.fit(features_train, target_train)

score_dummy = dummy.score(features_train, target_train)

print("Exactitud del modelo dummy: {}".format(score_dummy))

Exactitud del modelo dummy: 0.6924273858921162


DummyClassifier: 0.6924
RandomForestClassifier: 0.7854
Diferencia: 0.093 (9.3%)

Es un buen indicio que el modelo está funcionando correctamente.

Conclusión: El modelo Random Forest está funcionando. Aunque solo tenga 10 árboles, ya supera la predicción trivial del Dummy, lo que indica que está generalizando mejor y que sus features tienen valor predictivo.